Nikhil Adithyan
Aug 1, 2023 /7 min read

I’ve always been a huge fan of TradingView’s charting tool, especially for its beautifully crafted user interface and design. And there’s never been a day I haven’t thought about recreating the graph design in Python.
But, it’s always tough for Python developers to create stunning and professional-looking visualizations (like TradingView) using libraries like Matplotlib, Seaborn, Altair, etc. Their style themes are so outdated and crappy. Though all these modules provide features for customizing the theme of the charts, it takes a toll on the developer as there is a lot of work involved.


Fortunately, I recently came across an awesome library called lightweight-charts-python providing features to easily re-create the TradingView style with minimal code. In this article, we will dive deep into this library, explore its features, and code some cool TradingView charts in Python.

Importing Packages
The first and foremost step of setting up the coding environment is to import the required packages. In this article, we are going to use five different packages which are pandas for data manipulation, and requests for making API calls, numpy for numerical calculations, lightweight_chart for replicating the TradingView look, time for time-related functions, and finally asyncio and nest_asyncio for asynchronous programming. The following code will import all the mentioned packages into our Python environment:

In [1]:
import pandas as pd
import requests
import numpy as np
from lightweight_charts import Chart
from stock_indicators import indicators, Quote
from datetime import datetime, timedelta
import asyncio
import nest_asyncio
nest_asyncio.apply()


Obtaining Data using Yfinance

In [2]:
import yfinance as yf
df = yf.download('SPY', start='2020-01-01', multi_level_index=False)
df.reset_index(inplace=True)

df.head()

C:\Users\jwang\AppData\Local\Temp\ipykernel_20820\2552290532.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('SPY', start='2020-01-01', multi_level_index=False)
[*********************100%***********************]  1 of 1 completed


,Date,Close,High,Low,Open,Volume
0,2020-01-02,298.578613,298.597013,296.427990,297.356261,59151200
1,2020-01-03,296.317719,297.448186,295.113738,295.168880,77709700
2,2020-01-06,297.448181,297.530894,294.433601,294.553085,55653900
3,2020-01-07,296.611847,297.356292,296.161483,296.878357,40496400
4,2020-01-08,298.192596,299.414949,296.556651,296.804790,68296000


In [3]:
quotes = [
    Quote(d, o, h, l, c, v)
    for d, o, h, l, c, v in zip(
        df['Date'],
        df['Open'],
        df['High'],
        df['Low'],
        df['Close'],
        df['Volume']
    )
]

In [4]:
# Calculate EMA
df['EMA 12'] = [r.ema for r in indicators.get_ema(quotes, 12)]
df['EMA 20'] = [r.ema for r in indicators.get_ema(quotes, 20)]
df['EMA 25'] = [r.ema for r in indicators.get_ema(quotes, 25)]

# Calculate EMA12 crossover EMA25
df['bullishEMA'] = 0.0
df['bullishEMA'] = np.where(df['EMA 12'] > df['EMA 25'], 1.0, 0.0)
df['crossover_EMA12_EMA25'] = df['bullishEMA'].diff()

# Calculate Chandelier exit
from stock_indicators import ChandelierType 
df['chandelier_long_exit'] = [r.chandelier_exit for r in indicators.get_chandelier(quotes, 22, 3, ChandelierType.LONG)]
df['chandelier_short_exit'] = [r.chandelier_exit for r in indicators.get_chandelier(quotes, 22, 3, ChandelierType.SHORT)]

# Calculate Bollinger Bands
df['upper_band'] = [r.upper_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['middle_band'] = [r.sma for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['lower_band'] = [r.lower_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]

# Calculate RSI
df['rsi'] = [r.rsi for r in indicators.get_rsi(quotes, 14)]
df['rsima6'] = df['rsi'].rolling(6).mean()
df['rsima12'] = df['rsi'].rolling(12).mean()

# Calculate Mcginley dynamic
df['dynamic20'] = [r.dynamic for r in indicators.get_dynamic(quotes, 20)]


In [5]:
df = df.dropna().reset_index(drop=True)
df.head()

,Date,Close,High,Low,Open,Volume,EMA 12,EMA 20,EMA 25,bullishEMA,crossover_EMA12_EMA25,chandelier_long_exit,chandelier_short_exit,upper_band,middle_band,lower_band,rsi,rsima6,rsima12,dynamic20
0,2020-02-07,305.315491,306.960608,304.764042,305.885310,64139400,302.501862,301.768534,301.293821,1.0,0.0,298.431216,303.486936,308.087220,302.095946,296.104671,58.937505,54.794791,55.158688,301.480248
1,2020-02-10,307.594696,307.659038,304.387147,304.423918,42070000,303.285375,302.323406,301.778504,1.0,0.0,298.895728,303.537048,308.813358,302.508147,296.202936,62.231094,58.072141,54.571764,301.950462
2,2020-02-11,308.127869,309.745415,307.594792,308.955027,54864500,304.030374,302.876212,302.266916,1.0,0.0,301.087158,303.431996,309.580472,302.844069,296.107666,62.979159,60.647483,54.922732,302.425189
3,2020-02-12,310.112976,310.324344,309.203077,309.570699,43992700,304.966159,303.565428,302.870459,1.0,0.0,301.760125,303.337958,310.677135,303.302225,295.927315,65.702886,62.266580,56.606859,303.004634
4,2020-02-13,309.782166,310.756379,308.403559,308.679269,54501900,305.707083,304.157498,303.402129,1.0,0.0,302.260603,303.269514,311.556066,303.709836,295.863606,64.846667,62.878613,57.605057,303.521600


In [6]:
# Calculate the Chandelier Exit for both long and short positions
chandelier_long = indicators.get_chandelier(quotes, 22, 3, ChandelierType.LONG)
chandelier_short = indicators.get_chandelier(quotes, 22, 3, ChandelierType.SHORT)

# Convert the results to DataFrames
long_exit_df = df[["Date", "chandelier_long_exit" ]]

short_exit_df = df[["Date", "chandelier_short_exit" ]]

# Create a single dataframe to track the trend state
combined_df = df[['Date', 'Close']].copy()
combined_df = combined_df.merge(long_exit_df, on='Date', how='left', suffixes=('_close', '_long'))
combined_df = combined_df.merge(short_exit_df, on='Date', how='left', suffixes=('_long', '_short'))

# Initialize new columns for the segmented lines
combined_df['long_segment'] = combined_df['chandelier_long_exit']
combined_df['short_segment'] = combined_df['chandelier_short_exit']

# Create the list of data points with color and value
final_data = []
is_long_trend = False
for i in range(len(combined_df)):
    current_close = combined_df.loc[i, 'Close']
    current_long = combined_df.loc[i, 'chandelier_long_exit']
    current_short = combined_df.loc[i, 'chandelier_short_exit']

    # Skip initial NaN values
    if pd.isna(current_long) or pd.isna(current_short):
        continue

    # Determine initial trend
    if i == 0 or (pd.isna(combined_df.loc[i-1, 'Close'])):
        is_long_trend = (current_close > current_long)
    else:
        prev_close = combined_df.loc[i-1, 'Close']
        prev_long = combined_df.loc[i-1, 'chandelier_long_exit']
        prev_short = combined_df.loc[i-1, 'chandelier_short_exit']

        # Check for crossover
        if not is_long_trend and prev_close < prev_short and current_close > current_long:
            # Crossover from short to long
            final_data.append({'Date': combined_df.loc[i, 'Date'], 'value': None})
            is_long_trend = True
        elif is_long_trend and prev_close > prev_long and current_close < current_short:
            # Crossover from long to short
            final_data.append({'Date': combined_df.loc[i, 'Date'], 'value': None})
            is_long_trend = False

    # Add the current data point with the appropriate value and color
    if is_long_trend:
        final_data.append({
            'Date': combined_df.loc[i, 'Date'],
            'value': current_long,
            'color': 'green'
        })
    else:
        final_data.append({
            'Date': combined_df.loc[i, 'Date'],
            'value': current_short,
            'color': 'red'
        })


In [7]:
df.tail()

,Date,Close,High,Low,Open,Volume,EMA 12,EMA 20,EMA 25,bullishEMA,crossover_EMA12_EMA25,chandelier_long_exit,chandelier_short_exit,upper_band,middle_band,lower_band,rsi,rsima6,rsima12,dynamic20
1427,2025-10-13,663.039978,665.130005,659.770020,660.650024,79560500,664.929824,662.780622,661.008532,1.0,0.0,655.968998,670.472045,675.039781,664.424088,653.808394,52.632831,61.696227,64.458754,660.969042
1428,2025-10-14,662.229980,665.830017,653.169983,657.169983,88779600,664.514463,662.728180,661.102489,1.0,0.0,655.059948,671.381095,674.906986,664.626825,654.346664,51.837479,58.364143,63.524600,661.073322
1429,2025-10-15,665.169983,670.229980,658.929993,666.820007,81702600,664.615312,662.960733,661.415374,1.0,0.0,654.377680,672.063363,674.741741,665.017447,655.293153,54.523669,56.362266,62.667550,661.406378
1430,2025-10-16,660.640015,668.710022,657.109985,666.820007,110563300,664.003728,662.739712,661.355731,1.0,0.0,653.685508,672.755535,674.732794,665.027997,655.323200,49.905127,52.961186,61.236384,661.342217
1431,2025-10-17,664.390015,665.760010,658.140015,659.500000,96386200,664.063157,662.896884,661.589137,1.0,0.0,653.567532,673.222507,674.753061,665.062497,655.371933,53.422488,50.803362,59.935501,661.591572


In [8]:
final_data = pd.DataFrame(final_data)
final_data.head()

,Date,value,color
0,2020-02-07,298.431216,green
1,2020-02-10,298.895728,green
2,2020-02-11,301.087158,green
3,2020-02-12,301.760125,green
4,2020-02-13,302.260603,green


In the code, the reason for changing the column names is that lightweight_charts demands a specific naming structure to plot the data. Now that we have adequate data to work with, let’s make some cool visualizations.

In [9]:
if __name__ == '__main__':

    chart = Chart(title="Chandelier Exit", maximize=True)
    chart.legend(visible=True)
    chart.set(df)

    
    # Create a line series for the downward-trending (short) segment
    chandelier_exit_line = chart.create_line("value", color="red", style="solid", width=2)
    chandelier_exit_line.set(final_data)

    chart.show(block=True)


It just takes as little as three lines of code to create a graph in the look of TradingView’s charting platform. And the code is very straightforward in nature. We are first creating an instance of the class Chart and assigned it to the chart variable. Then using the set function, we are setting the data of the chart. Finally, we are displaying the created chart with the help of the show function. 

This output is absolutely stunning for a program of three lines of code. But people who are disappointed at the output after viewing the thumbnail of this article, don't worry! Because now, we are going to up the game by customizing the whole theme of the plot, adding more details, and simply, making it even more beautiful. Here’s the code to do that:

In [10]:
if __name__ == '__main__':

    chart = Chart(title="TQQQ Stock Price", height = 600, width = 1000, maximize=True)

    chart.grid(vert_enabled = True, horz_enabled = True)

    chart.layout(background_color='#131722', font_family='Trebuchet MS', font_size = 16)

    chart.candle_style(up_color='#2962ff', down_color='#e91e63',
                    border_up_color='#2962ffcb', border_down_color='#e91e63cb',
                    wick_up_color='#2962ffcb', wick_down_color='#e91e63cb')

    chart.volume_config(up_color='#2962ffcb', down_color='#e91e63cb')

    chart.legend(visible = True, font_family = 'Trebuchet MS', ohlc = True, percent = True)

    #####################################################################################

    chart.set(df)

    # Create line series for EMAs
    ema12_line = chart.create_line('EMA 12', color='#ffeb3b', width=1, price_label=True)
    ema12_line.set(df[['Date', 'EMA 12']])

    ema25_line = chart.create_line('EMA 25', color='#26c6da', width=1, price_label=True)
    ema25_line.set(df[['Date', 'EMA 25']])

    chart.show(block = True)

The code might not be as short as the previous one for the basic plot, but it’s actually very simple. And for easy explanation, I’ve divided the code into two separate parts. The first part is about theme customization. It includes changing the background color, the colors of the candles and volume bars, and so on. Basically, the things related to the style of the plot are dealt with in the first part. The second part is about adding details to the plot. 

This is absolutely fantastic! We fully customized the whole look and feel of the plot and added more details like SMA lines and legends for a more insightful graph.


Now let’s move our focus from historical graphs to another cool feature provided by the lightweight_charts library which is the real-time charting feature. Real-time charts are extremely useful for day traders to keep track of the latest price movements and TradingView is most preferred for such charts. Just like how we replicated the historical charts of TradingView, let’s do the same thing for real-time charts too. This is the code to create a real-time TradingView chart:

In this code, we are not actually using the real-time data of stock prices but rather simulating it using the previously extracted historical data. We are first splitting the historical data into two separate dataframes. The first one is used as the initial data for the plot and the second one is used as the real-time data which is done by updating the data points of the plot with the help of a for-loop. 

Pretty cool, right?! But like how there was a lot of scope for improvements in the basic historical graph, this real-time chart can also be improved and modified in a lot of places. We can first change the theme of the plot and similar to how we added SMA lines to the historical chart for better insights, we can add more details for an informative visualization. Here’s the code for the modified or advanced version of the initial real-time chart:

In [11]:
from lightweight_charts import Chart
import pandas as pd

# Assuming `df` is already a pandas DataFrame with 'Date', 'Open', 'High', 'Low', 'Close', 'EMA 12', and 'EMA 25' columns.
# It's good practice to convert the 'Date' column to the correct datetime format.
# df['Date'] = pd.to_datetime(df['Date'])

if __name__ == '__main__':
    
    rt_chart = Chart( maximize=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    rt_chart.set(df)

    # Create line series for EMAs
    ema12_line = rt_chart.create_line('EMA 12', color='#ffeb3b', width=1, price_label=True)
    ema12_line.set(df[['Date', 'EMA 12']])

    ema25_line = rt_chart.create_line('EMA 25', color='#26c6da', width=1, price_label=True)
    ema25_line.set(df[['Date', 'EMA 25']])

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        emadiff = df.iloc[i]['crossover_EMA12_EMA25']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if emadiff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif emadiff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        rt_chart.marker_list(markers)
    
rt_chart.show(block = True)


In [12]:
if __name__ == '__main__':

    chart = Chart(title="Mcginley Dynamic 20", maximize=True)
    chart.legend(visible=True)
    chart.set(df)

    mcginley_line = chart.create_line('dynamic20', color='#26c6da', width=1, price_label=True)
    mcginley_line.set(df[['Date', 'dynamic20']])

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):
        p_mcginley, p_close = df.iloc[i-1]['dynamic20'], df.iloc[i-1]['Close']
        c_mcginley, c_close = df.iloc[i]['dynamic20'], df.iloc[i]['Close']
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if p_close < p_mcginley and c_close > c_mcginley :
            
        # Calculate the time for the next day
            next_day = current_time + timedelta(days=1)
            
            markers.append({
                'time': next_day,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif p_close > p_mcginley and c_close < c_mcginley :
             
             next_day = current_time + timedelta(days=1)
             
             markers.append({
                'time': next_day,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
    chart.show(block = True)

In [13]:
if __name__ == '__main__':
    
    chart = Chart(title="RSI crossove RSIMA12 + Mcginley dynamic 20", inner_width=1, inner_height=0.8, maximize=True)
    chart.legend(visible=True)
    chart.set(df)
    
    mcginley_line = chart.create_line('dynamic20', color="#e4b713", width=1, price_label=True, price_line=False)
    mcginley_line.set(df[['Date', 'dynamic20']])

    # chandelier_long_line = chart.create_line('chandelier_long_exit', color="#ec520b", width=1, price_label=False)
    # chandelier_long_line.set(df[['Date', 'chandelier_long_exit']])

    # chandelier_short_line = chart.create_line('chandelier_short_exit', color='#26c6da', width=1, price_label=False)
    # chandelier_short_line.set(df[['Date', 'chandelier_short_exit']])

    chart2 = chart.create_subchart(width=1, height=0.2, sync=True)
    rsi_line = chart2.create_line('rsi', color='green', width=1, price_line=False)
    df2 = df[['Date', 'rsi']]
    rsi_line.set(df2)
    
    rsima_line = chart2.create_line('rsima12', color='red', width=1, price_line=False)
    df3 = df[['Date', 'rsima12']]
    rsima_line.set(df3)

    # Initialize a list to hold the dynamic markers
    markers = []
    buy_signal = 0
    sell_signal = 0

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):
        p_mcginley, p_close = df.iloc[i-1]['dynamic20'], df.iloc[i-1]['Close']
        c_mcginley, c_close = df.iloc[i]['dynamic20'], df.iloc[i]['Close']
        p_rsi, p_rsima12 = df.iloc[i-1]['rsi'], df.iloc[i-1]['rsima12']
        c_rsi, c_rsima12 = df.iloc[i]['rsi'], df.iloc[i]['rsima12']
        c_chandelier_long = df.iloc[i]['chandelier_long_exit']
        c_chandelier_short = df.iloc[i]['chandelier_short_exit']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal
        if p_close < p_mcginley and c_close > c_mcginley and p_rsi < p_rsima12 and c_rsi > c_rsima12 and buy_signal == 0:
            next_day = current_time + timedelta(days=1)
            markers.append({
                'time': next_day,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
            buy_signal = 1
            sell_signal = 0

        # Check for sell signal
        elif p_close > p_mcginley and c_close < c_mcginley and p_rsi > p_rsima12 and c_rsi < c_rsima12 and sell_signal == 0:
            next_day = current_time + timedelta(days=1)
            markers.append({
                'time': next_day,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })
            sell_signal = 1
            buy_signal = 0

        # if buy_signal == 1 and c_close < c_chandelier_long:
        #     markers.append({
        #         'time': current_time,
        #         'position': 'above',
        #         'shape': 'arrow_down',
        #         'color': '#f485fb',
        #         'text': 'Long stop'
        #     })
        #     buy_signal = 0

        # elif sell_signal == 1 and c_close > c_chandelier_short:
        #     markers.append({
        #         'time': current_time,
        #         'position': 'below',
        #         'shape': 'arrow_up',
        #         'color': '#f485fb',
        #         'text': 'Short stop'
        #     })
        #     sell_signal = 0

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers) 
        
  
    chart.show(block = True)